In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, BooleanType, IntegerType
from pyspark import *

In [2]:
from pyspark.sql import SparkSession

# Инициализируем sparkContext
spark = SparkSession.builder \
    .master('local') \
    .appName('Dreamlone') \
    .config('spark.executor.memory', '4gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

sc = spark.sparkContext

# Используем SQLContext чтобы прочитать файл
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Task 6

### Find fans/lovers. The idea is, if the user likes another posts, and its is not mutual, then the first fan/lover.

In [3]:
likes = sqlContext.read.parquet('./bigdata20/followers_posts_likes.parquet')
likes_agg = likes.groupBy('likerId').agg(collect_set('ownerId').alias('who_liked'))

likes_agg = likes_agg.selectExpr('likerId as liker_guy', 'who_liked as who_liked')
joined_data = likes.join(likes_agg, likes.ownerId == likes_agg.liker_guy)                                                        

# Создаем поле (столбец) Friends в котором содержится информация о том, являются ли пользователи друзьями
friends = joined_data.withColumn("Friends", expr("array_contains(who_liked, likerId)"))
fans = friends.filter(friends.Friends == False)

fans.groupBy('ownerId').agg(collect_set('likerId').alias('fans')) \
.write.format("json").save("./bigdata20/Answers/task6_fans.json")